In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
def average_embeddings(Xmat):
    return np.mean(Xmat, axis=1)

Xmat_train = # shape: (num_examples_train, 64, 50)
Y_train = 

Xmat_test = # shape: (num_examples_test, 64, 50)
Y_test =

# Average the word embeddings for each example into a sentence embedding
X_train_avg = average_embeddings(Xmat_train)
X_test_avg = average_embeddings(Xmat_test)

# Train a Logistic Regression model using the averaged embeddings
baseline_embed = LogisticRegression(max_iter=1000, multi_class='auto', solver='lbfgs')
baseline_embed.fit(X_train_avg, Y_train)

# Prediction & Evaluation
Y_pred_train = baseline_embed.predict(X_train_avg)
Y_pred_test = baseline_embed.predict(X_test_avg)

train_accuracy = accuracy_score(Y_train, Y_pred_train)
test_accuracy = accuracy_score(Y_test, Y_pred_test)

print(f"Train accuracy: {train_accuracy}")
print(f"Test accuracy: {test_accuracy}")


SyntaxError: invalid syntax (2950157888.py, line 4)